# cut cross entropy

author: [xiaodongguaAIGC](https://github.com/dhcode-cpp)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## Cross-Entropy

In [2]:
X = torch.tensor([1.0, 3.0, -1.2, 1.1, -0.5, -0.8]) # logits
Y = torch.tensor([0.4, 0.2, 0.1,  0.1, 0.05, 0.15])

### Cross Entropy Pytorch

$CE(X,Y) = \sum_i -\log(\text{softmax}(X))$

In [3]:
loss = nn.CrossEntropyLoss()
l = loss(X,Y)
print(l)

tensor(2.4569)

### Cross Entropy with log softmax

when we calculate cross entropy, we need logprob

In [4]:
l = -Y * F.log_softmax(X, dim = 0 )
l = l.sum()
print(l)

tensor(2.4569)

In [5]:
p = F.softmax(X, dim = 0 )
l = -Y * p.log()
l = l.sum()
print(l)

tensor(2.4569)

## Softmax

### forward

In [6]:
p = torch.exp(X) / (torch.exp(X)).sum()
print(p)
l = - Y * p.log()
l = l.sum()
print(l)

tensor([0.1001, 0.7394, 0.0111, 0.1106, 0.0223, 0.0165])

tensor(2.4569)

### gradient

In [7]:
N = 6 
dX = (p - Y) / 6
print(dX)

tensor([-0.0500,  0.0899, -0.0148,  0.0018, -0.0046, -0.0222])

In [8]:
dpdX = torch.diag(p) - torch.outer(p, p)
dX = -(Y / p) @ dpdX   # first term is (- Y log p)' = (-Y/p)
print(dX)

tensor([-0.2999,  0.5394, -0.0889,  0.0106, -0.0277, -0.1335])

## Log Softmax

### forward

In [9]:
LSE = torch.log( torch.sum( torch.exp( X ) ) )
p = X - LSE # X - LSE
print(p)
l = - Y * p # delete log
l = l.sum()
print(l)

tensor([-2.3019, -0.3019, -4.5019, -2.2019, -3.8019, -4.1019])

tensor(2.4569)

### gradient

首先，L = -∑y_i * log(softmax(x_i))

= -∑y_i * p_i (因为p_i = LogSoftmax(x_i))

求导链式法则：

∂L/∂x_k = ∑_i (∂L/∂p_i * ∂p_i/∂x_k)

其中：

∂L/∂p_i = -y_i

∂p_i/∂x_k = δ_{ik} - softmax(x_k) 

代入得到：

∂L/∂x_k = ∑_i (-y_i * (δ_{ik} - softmax(x_k)))

= -y_k + ∑_i y_i * softmax(x_k)

= -y_k + softmax(x_k) * ∑_i y_i

= softmax(x_k) - y_k (因为∑y_i = 1)


In [10]:
l = - Y * p / N
# p.exp() is softmax result probability
dX = p.exp() - Y # lse is easy revearse p 
print(dX)

tensor([-0.2999,  0.5394, -0.0889,  0.0106, -0.0277, -0.1335])

## Cut Cross Entropy

think about llm model has a big vocabulary, so the classifer project d-dimension to |V|-dimension, and |V| usually is big like 128,000 or 256,000.

classifier proj is big GEMM when the model params < 2B

In [11]:
V = 128
D = 16
BS = 8
block_size = 4
n_b = BS // block_size
d_b = D // block_size
v_b = V // block_size

print(n_b)
print(d_b)
print(v_b)

2

4

32

In [12]:
C = torch.randn(D, V)
E = torch.randn(BS, D)
Y = torch.randn(BS, V)
Y = torch.softmax(Y, dim = 1)
print(Y.shape) # label

torch.Size([8, 128])

### standard CE

In [13]:
# ce
logits = E @ C 
print(logits.shape)
print(Y.shape)
l = loss(logits, Y)
print(l)

torch.Size([8, 128])

torch.Size([8, 128])

tensor(10.8182)

In [15]:
# softmax -> ce
P = F.softmax(logits, dim = 1)
print(P)
l = -Y * P.log()
print(l.sum() / (BS*V))

tensor([[1.6443e-05, 2.1251e-03, 2.7414e-05,  ..., 9.0985e-06, 1.8049e-03,
         5.5270e-05],
        [2.0362e-03, 2.1562e-03, 4.0625e-03,  ..., 3.1830e-05, 5.5824e-04,
         2.2409e-03],
        [8.3387e-08, 7.1920e-05, 8.6115e-11,  ..., 1.4612e-04, 2.5758e-05,
         4.5846e-05],
        ...,
        [6.4367e-05, 2.8981e-04, 6.4149e-06,  ..., 5.1006e-04, 3.6821e-08,
         7.0023e-06],
        [2.8601e-04, 9.0634e-05, 9.9731e-04,  ..., 7.0501e-09, 1.4259e-04,
         2.5107e-06],
        [5.9795e-06, 7.6840e-06, 2.2043e-03,  ..., 4.0103e-07, 1.7927e-04,
         2.7438e-07]])

tensor(0.0845)

In [17]:
# logsoftmax -> ce
P = F.log_softmax(logits, dim = 1)
l = -Y * P
print(l.sum() / (BS*V))

tensor(0.0845)

### Cut Cross Entropy 

#### *Algorithm 1: forward GEMM*

our target is get  


logprob = X @ C - log \sum_j X[i,:] C[:,j] # row sub

the X[i,1] C[1,j] + X[i,2] C[2,j] + X[i,d] C[d,j] + 

output = -Y * logprob

In [18]:
X = E @ C

In [19]:
print(X.shape)

torch.Size([8, 128])

In [20]:
X_blocks = torch.split(X, 64, dim = 1)
print(X_blocks[0].shape)
print(len(X_blocks))

torch.Size([8, 64])

2

In [21]:
E_blocks = torch.split(E, n_b, dim = 0)
C_blocks = torch.split(C, v_b, dim = 1)

print(len(E_blocks), len(C_blocks))
print(E_blocks[0].shape)
print(C_blocks[0].shape)

4 4

torch.Size([2, 16])

torch.Size([16, 32])

In [22]:
stand_mm = E @ C
print(stand_mm[:5,:5])

tensor([[ -0.4785,   4.3832,   0.0327,  -4.4717,  -3.0880],
        [  2.2906,   2.3479,   2.9814,  -1.3908,   0.1317],
        [ -4.6312,   2.1287, -11.5067,  -1.8696,  -4.7591],
        [  0.2307,  -5.2615,  -2.6144,   3.1511,   4.0318],
        [ -2.8655,  -3.7288,   0.5967,  -2.9989,   4.1903]])

In [24]:
# CCE : Algorithm 1

O = torch.zeros(BS, V)
O_blocks = torch.split(O, n_b, dim = 0)
O_result = []

for e, o in zip(E_blocks, O_blocks): # i
    e_d_blocks = torch.split(e, d_b, dim = 1)
    c_blocks = torch.split(C, d_b, dim = 0)
    for ed, cd in zip(e_d_blocks, c_blocks): # 
        a = ed @ cd
        o = o + a
    O_result.append(o)

O_mat = torch.cat(O_result, dim = 0)
print(O_mat.shape)

print(O_mat[:5,:5])

torch.Size([8, 128])

tensor([[ -0.4785,   4.3832,   0.0327,  -4.4717,  -3.0880],
        [  2.2906,   2.3479,   2.9814,  -1.3908,   0.1317],
        [ -4.6312,   2.1287, -11.5067,  -1.8696,  -4.7591],
        [  0.2307,  -5.2615,  -2.6144,   3.1511,   4.0318],
        [ -2.8655,  -3.7288,   0.5967,  -2.9989,   4.1903]])

#### *Algorithm 2: forward logsoftmax*

Original LSE

In [25]:
A = E @ C 
LSE = torch.log( torch.sum( torch.exp(A), dim = 1, keepdim=True ) )
print(LSE)

tensor([[10.5371],
        [ 8.4873],
        [11.6686],
        [10.2317],
        [11.1235],
        [12.1996],
        [10.9306],
        [11.2929]])

In [27]:
# CCE : Algorithm 2

LSE = torch.ones(BS, 1) * -100000.0 # -inf

# LSE part
for n, e in enumerate(E_blocks):
    for v, c in enumerate(C_blocks):
        A = torch.zeros(n_b, v_b)
        e_d_blocks = torch.split(e, d_b, dim = 1)
        c_d_blocks = torch.split(c, d_b, dim = 0)
        for ed, cd in zip(e_d_blocks, c_d_blocks):
            A += ed @ cd
        LSE_nv = torch.log( torch.sum( torch.exp(A), dim = 1, keepdim=True ) )
        LSE[n_b * n : n_b * (n+1)] = torch.log( torch.exp(LSE[n_b * n : n_b * (n+1)]) 
                                                    + torch.exp(LSE_nv))
print(LSE.shape)
print(LSE)

torch.Size([8, 1])

tensor([[10.5371],
        [ 8.4873],
        [11.6686],
        [10.2317],
        [11.1235],
        [12.1996],
        [10.9306],
        [11.2929]])

#### Backward

#### standard backward

In [29]:
# basic backward
# P = softmax(E @ C, dim = 1)
# loss = - Y * torch.log(P)

Y = torch.randn(BS, V)
Y = torch.softmax(Y, dim = 1)

X = E @ C
P = torch.softmax(X, dim = 1)

loss = (- Y * torch.log(P)).mean()
print(loss)

tensor(0.0863)

In [32]:
# Stand backward
N = BS * V

dP = (P - Y) / N
dPdX = torch.ones(V, V)
for i in range(BS):
    dPdX += torch.diag(P[i,:]) - torch.outer(P[i,:], P[i,:]) 

dX = dP @ dPdX 
print(dX.shape)

dE = dP @ C.t()
dC = E.t() @ dP 

print(dE[:5,:5])
print(dC[:5,:5])


torch.Size([8, 128])

tensor([[-2.6092e-04,  3.0870e-04,  6.9659e-05, -4.3250e-04, -1.4923e-03],
        [-1.4229e-03,  8.1375e-04, -2.5839e-04, -9.5139e-05, -1.5609e-04],
        [-7.6372e-05, -1.1756e-03, -9.7960e-04,  8.9844e-04,  8.6361e-04],
        [ 2.6368e-04, -7.8118e-04,  3.4700e-04,  4.2098e-04,  8.1858e-04],
        [-4.6616e-04, -8.9668e-04,  3.9459e-04,  6.4474e-04,  4.2903e-04]])

tensor([[ 2.4710e-05,  1.1704e-05,  1.8064e-05,  3.0421e-05,  1.9202e-05],
        [-1.9689e-05, -9.9801e-06, -3.2482e-06, -1.0947e-05, -6.9049e-06],
        [-2.0659e-06, -6.2992e-06, -2.2300e-05,  2.5356e-06, -8.4896e-06],
        [ 9.3127e-06,  2.6459e-06,  9.4267e-06, -9.3367e-10,  5.9976e-06],
        [-3.4611e-05, -2.3142e-05, -3.1973e-05, -8.8207e-06,  8.1728e-06]])

#### LSE Backward

In [33]:
X = E @ C
LSE = torch.log(torch.sum( torch.exp(X), dim = 1, keepdim = True))
print(LSE)
P = X - LSE # 逐行减去LSE
loss = (- Y * P) # (-Y * X + Y * LSE)
print(loss.mean() )


tensor([[10.5371],
        [ 8.4873],
        [11.6686],
        [10.2317],
        [11.1235],
        [12.1996],
        [10.9306],
        [11.2929]])

tensor(0.0863)

In [34]:
dY_dX = -Y / N # P = "X" - LSE
dY_dLSE = torch.sum(Y / N, dim = 1, keepdim = True) # P = X - "LSE"
dLSE_dX = F.softmax(X, dim = 1)
dY_dX_LSE = dY_dLSE * dLSE_dX

dY_dX_total =  dY_dX + dY_dX_LSE

dE = dY_dX_total @ C.t()
dC = E.t() @ dY_dX_total 

print(dE[:5,:5])
print(dC[:5,:5])

tensor([[-2.6092e-04,  3.0870e-04,  6.9659e-05, -4.3250e-04, -1.4923e-03],
        [-1.4229e-03,  8.1375e-04, -2.5839e-04, -9.5139e-05, -1.5609e-04],
        [-7.6372e-05, -1.1756e-03, -9.7960e-04,  8.9844e-04,  8.6361e-04],
        [ 2.6368e-04, -7.8118e-04,  3.4700e-04,  4.2098e-04,  8.1858e-04],
        [-4.6616e-04, -8.9668e-04,  3.9459e-04,  6.4474e-04,  4.2903e-04]])

tensor([[ 2.4710e-05,  1.1704e-05,  1.8064e-05,  3.0421e-05,  1.9202e-05],
        [-1.9689e-05, -9.9801e-06, -3.2482e-06, -1.0947e-05, -6.9049e-06],
        [-2.0659e-06, -6.2992e-06, -2.2300e-05,  2.5356e-06, -8.4896e-06],
        [ 9.3127e-06,  2.6459e-06,  9.4267e-06, -9.3367e-10,  5.9976e-06],
        [-3.4611e-05, -2.3142e-05, -3.1973e-05, -8.8207e-06,  8.1728e-06]])

In [35]:
dE_LSE = dLSE_dX @ C.t()
dC_LSE = E.t() @ dLSE_dX 
print(dE_LSE[:5,:5])
print(dC_LSE[:5,:5])

tensor([[-0.1516,  0.4684,  0.1270, -0.4346, -1.5616],
        [-1.4657,  0.9885, -0.3458, -0.2724, -0.1740],
        [ 0.0425, -0.9336, -0.8766,  0.9913,  1.0803],
        [ 0.2894, -0.7057,  0.3955,  0.3473,  0.7951],
        [-0.2545, -0.6627,  0.5302,  0.5925,  0.3672]])

tensor([[-0.0022, -0.0029, -0.0095,  0.0006,  0.0025],
        [ 0.0012,  0.0012,  0.0050, -0.0010, -0.0030],
        [-0.0005, -0.0008, -0.0022,  0.0014,  0.0046],
        [-0.0006, -0.0011, -0.0012, -0.0003, -0.0005],
        [ 0.0001, -0.0026,  0.0039,  0.0006,  0.0020]])

#### Pytorch AutoGrad LSE backward

 仅检验手动计算的梯度是否正确

In [36]:
# # 注意要清除梯度，得到的才准确
# # E.grad.zero_()
# # C.grad.zero_()
# # Y.grad.zero_()

# E.requires_grad = True
# C.requires_grad = True
# Y.requires_grad = True


# X = E @ C
# X.retain_grad()

# LSE = torch.log(torch.sum( torch.exp(X), dim = 1, keepdim = True))
# LSE.retain_grad()

# P = X - LSE # 逐行减去LSE
# P.retain_grad()

# loss = (- Y * P) # (-Y * X + Y * LSE)
# loss.retain_grad()
# # print(loss)

# l = loss.mean()
# l.backward() 

In [37]:
# print(loss.grad[:5,:5])

# print(LSE.grad[:5,:5])
# print(LSE.grad.shape)

# print(X.grad[:5,:5])
# print(E.grad[:5,:5])
# print(C.grad[:5,:5])

#### Algorithm 3: CCE LSE Backward

In [38]:
# CCE : Algorithm 3

is_filter = False
epsilon = 1e-6

dE = torch.zeros_like(E)
dC = torch.zeros_like(C)

dE_blocks = torch.split(dE, n_b, dim = 0)
dC_blocks = torch.split(dC, v_b, dim = 1)

print(Y.shape)
d_LSE = dY_dLSE
print(d_LSE)

# LSE part
for n, e in enumerate(E_blocks):
    for v, c in enumerate(C_blocks):
        
        A = torch.zeros(n_b, v_b)
        e_d_blocks = torch.split(e, d_b, dim = 1)
        c_d_blocks = torch.split(c, d_b, dim = 0)
        for ed, cd in zip(e_d_blocks, c_d_blocks):
            A += ed @ cd # A_nv

        S = torch.exp(A - LSE[n_b * n : n_b * (n+1)]) # softmax

        # filter S
        if is_filter:
            if  (S < epsilon).all():
                print('skip', n, v)
                continue
            
        
        d_LSE_block = d_LSE[n_b * n : n_b * (n+1), :]
        d_index = 0
        for ed, cd in zip(e_d_blocks, c_d_blocks):
            dE_blocks[n][ :, d_index * d_b: (d_index+1)*d_b ] +=  (d_LSE_block * S) @ cd.t()
            dC_blocks[v][ d_index * d_b: (d_index+1)*d_b, : ] +=  ed.t() @ (d_LSE_block * S)
            d_index = d_index + 1


# dLSE gradient part
dE_part_LSE = torch.cat(dE_blocks, dim = 0)
dC_part_LSE = torch.cat(dC_blocks, dim = 1)

print(dE_part_LSE.shape)
print(dC_part_LSE.shape)

print(dE_part_LSE[:5,:5])
print(dC_part_LSE[:5,:5])

torch.Size([8, 128])

tensor([[0.0010],
        [0.0010],
        [0.0010],
        [0.0010],
        [0.0010],
        [0.0010],
        [0.0010],
        [0.0010]])

torch.Size([8, 16])

torch.Size([16, 128])

tensor([[-1.4807e-04,  4.5747e-04,  1.2400e-04, -4.2441e-04, -1.5250e-03],
        [-1.4313e-03,  9.6535e-04, -3.3768e-04, -2.6606e-04, -1.6991e-04],
        [ 4.1456e-05, -9.1171e-04, -8.5606e-04,  9.6804e-04,  1.0550e-03],
        [ 2.8259e-04, -6.8914e-04,  3.8623e-04,  3.3917e-04,  7.7647e-04],
        [-2.4849e-04, -6.4716e-04,  5.1773e-04,  5.7858e-04,  3.5864e-04]])

tensor([[-2.1446e-06, -2.8563e-06, -9.2369e-06,  6.3201e-07,  2.4046e-06],
        [ 1.2073e-06,  1.1698e-06,  4.8543e-06, -9.3262e-07, -2.8891e-06],
        [-5.3356e-07, -7.4234e-07, -2.1108e-06,  1.4130e-06,  4.4816e-06],
        [-5.8942e-07, -1.0355e-06, -1.1461e-06, -2.7859e-07, -4.9734e-07],
        [ 1.1891e-07, -2.5827e-06,  3.8380e-06,  5.3986e-07,  1.9357e-06]])

In [39]:
# dX part dE, dC
dE_part_X = dY_dX @ C.t()
dC_part_X = E.t() @ dY_dX

# dX Total
dE_total = dE_part_X + dE_part_LSE
dC_total = dC_part_X + dC_part_LSE

print(dE_total[:5, :5])
print(dC_total[:5, :5])

tensor([[-2.6092e-04,  3.0870e-04,  6.9659e-05, -4.3250e-04, -1.4923e-03],
        [-1.4229e-03,  8.1375e-04, -2.5839e-04, -9.5139e-05, -1.5609e-04],
        [-7.6372e-05, -1.1756e-03, -9.7960e-04,  8.9844e-04,  8.6361e-04],
        [ 2.6368e-04, -7.8118e-04,  3.4700e-04,  4.2098e-04,  8.1858e-04],
        [-4.6616e-04, -8.9668e-04,  3.9459e-04,  6.4474e-04,  4.2903e-04]])

tensor([[ 2.4710e-05,  1.1704e-05,  1.8064e-05,  3.0421e-05,  1.9202e-05],
        [-1.9689e-05, -9.9801e-06, -3.2482e-06, -1.0947e-05, -6.9049e-06],
        [-2.0659e-06, -6.2992e-06, -2.2300e-05,  2.5356e-06, -8.4896e-06],
        [ 9.3127e-06,  2.6459e-06,  9.4267e-06, -9.3405e-10,  5.9977e-06],
        [-3.4611e-05, -2.3142e-05, -3.1973e-05, -8.8207e-06,  8.1728e-06]])

#### Algorithm 4: CCE Backward total

In [40]:
print(Y)
Y_t = Y / 0.001 # be sharp
YY = F.softmax(Y_t, dim = 1)
print(YY)
_, label = torch.max(YY, dim = 1)
print(label)

print(YY[0, label[0]])

tensor([[0.0017, 0.0032, 0.0010,  ..., 0.0034, 0.0128, 0.0054],
        [0.0051, 0.0039, 0.0121,  ..., 0.0054, 0.0009, 0.0035],
        [0.0049, 0.0103, 0.0031,  ..., 0.0047, 0.0064, 0.0201],
        ...,
        [0.0103, 0.0132, 0.0067,  ..., 0.0077, 0.0109, 0.0348],
        [0.0007, 0.0008, 0.0075,  ..., 0.0048, 0.0021, 0.0069],
        [0.0153, 0.0067, 0.0129,  ..., 0.0215, 0.0102, 0.0016]])

tensor([[8.5727e-19, 3.9012e-18, 4.2295e-19,  ..., 4.6580e-18, 5.5371e-14,
         3.4361e-17],
        [2.8504e-27, 8.6260e-28, 3.0629e-24,  ..., 3.7687e-27, 4.0765e-29,
         5.6033e-28],
        [9.5664e-37, 2.0436e-34, 1.5887e-37,  ..., 7.7930e-37, 4.1209e-36,
         3.7760e-30],
        ...,
        [2.0265e-16, 3.6451e-15, 5.8259e-18,  ..., 1.5183e-17, 3.7494e-16,
         9.3810e-06],
        [8.0915e-36, 9.5897e-36, 7.4534e-33,  ..., 5.2601e-34, 3.5684e-35,
         4.0869e-33],
        [1.1188e-14, 1.9191e-18, 9.5395e-16,  ..., 5.5804e-12, 6.7919e-17,
         1.2096e-20]])

tensor([ 46,  11,  27,  18, 121,  47,  79,  12])

tensor(1.0000)

##### max likelihood version CE

In [41]:
print(P.shape)

torch.Size([8, 128])

In [42]:
print(label.unsqueeze(0))
P_CEL = torch.gather(P, dim = 1, index = label.unsqueeze(1))
loss = -P_CEL
l = loss.mean()
print(loss)
print(l)

tensor([[ 46,  11,  27,  18, 121,  47,  79,  12]])

tensor([[13.7104],
        [10.2666],
        [15.8899],
        [ 7.8233],
        [ 8.1101],
        [14.2360],
        [12.2592],
        [18.1864]])

tensor(12.5602)

In [43]:
loss = nn.CrossEntropyLoss()
l_torch = loss(X, label)
print(l_torch)

tensor(12.5602)

In [44]:
# CCE : Algorithm 4

bs = 8
v = 128


M = torch.zeros(bs, v)
for i in range(bs):
    M[i, label[i]] = 1
M_blocks = torch.split(M, n_b, dim = 0)

dE = torch.zeros_like(E)
dC = torch.zeros_like(C)

dE_blocks = torch.split(dE, n_b, dim = 0)
dC_blocks = torch.split(dC, v_b, dim = 1)

d_LCE = torch.ones(bs, 1) / bs

# LSE part
for n, e in enumerate(E_blocks):
    
    M_v_blcoks = torch.split(M_blocks[n], v_b, dim = 1)
                                 
    for v, c in enumerate(C_blocks):
        
        A = torch.zeros(n_b, v_b)
        e_d_blocks = torch.split(e, d_b, dim = 1)
        c_d_blocks = torch.split(c, d_b, dim = 0)
        for ed, cd in zip(e_d_blocks, c_d_blocks):
            A += ed @ cd # A_nv


        S = torch.exp(A - LSE[n_b * n : n_b * (n+1)]) # softmax
        G = - M_v_blcoks[v] + S
        
        
        # filter S
        if is_filter:
            if  (S < epsilon).all():
                print('skip',n,v)
                continue
        
        d_LCE_block = d_LCE[n_b * n : n_b * (n+1), :]
        d_index = 0
        for ed, cd in zip(e_d_blocks, c_d_blocks):
            dE_blocks[n][ :, d_index * d_b: (d_index+1)*d_b ] +=  (d_LCE_block * G) @ cd.t()
            dC_blocks[v][ d_index * d_b: (d_index+1)*d_b, : ] +=  ed.t() @ (d_LCE_block * G)
            d_index = d_index + 1


# dLSE gradient part
dE = torch.cat(dE_blocks, dim = 0)
dC = torch.cat(dC_blocks, dim = 1)

print(dE[:5,:5])
print(dC[:5,:5])

tensor([[-0.1959,  0.0150,  0.1927, -0.0049, -0.1715],
        [-0.4244,  0.0286, -0.0323,  0.3292, -0.0088],
        [-0.1098, -0.2441, -0.1873,  0.1830,  0.0889],
        [ 0.1286, -0.1850,  0.0862,  0.0316,  0.1351],
        [ 0.0431, -0.0286,  0.0516,  0.2682,  0.1283]])

tensor([[-2.7451e-04, -3.6561e-04, -1.1823e-03,  8.0897e-05,  3.0779e-04],
        [ 1.5454e-04,  1.4973e-04,  6.2136e-04, -1.1937e-04, -3.6980e-04],
        [-6.8296e-05, -9.5019e-05, -2.7018e-04,  1.8086e-04,  5.7365e-04],
        [-7.5446e-05, -1.3255e-04, -1.4670e-04, -3.5659e-05, -6.3659e-05],
        [ 1.5220e-05, -3.3058e-04,  4.9126e-04,  6.9102e-05,  2.4777e-04]])